In [19]:
using Revise
using POMDPs
using DiscreteValueIteration
using POMDPTools
using SubmodularMaximization
# using Dates
# using Profile

In [10]:
using MDMA

In [11]:
# Experiment name
# This must match the blend file name and will name the folder 
# for which experiment data is saved in /output
experiment_name="spreadout_group"
# Set model parameters
focal_length = [19.99, 19.99]#mm
resolution = [3840.0,2160.0]
lens_dim = [35.00,24.00] #mm
pitch = -pi/2
cutoff = 5.
sensor = MDMA.PinholeCameraModel(focal_length, resolution, lens_dim, 0.0, pitch, cutoff) 
move_dist = 3;
multi_configs = configs_from_file("../../blender/$(experiment_name)_data.json",experiment_name, move_dist)
# model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, target_trajectories, move_dist);
robot_states = map(x->MDMA.random_state(multi_configs.horizon, multi_configs.grid), 1:multi_configs.num_robots)
robot_states = map(x->MDMA.random_state(multi_configs.horizon, multi_configs.grid), 1:multi_configs.num_robots)
multi_problem = MDMA.MultiRobotTargetCoverageProblem(robot_states, multi_configs)

MultiRobotTargetCoverageProblem(MDPState[MDPState(UAVState(12, 14, :NE), 1, 60, nothing), MDPState(UAVState(28, 10, :N), 1, 60, nothing), MDPState(UAVState(15, 29, :NW), 1, 60, nothing)], MultiDroneMultiActorConfigs("spreadout_group", 3, Target[Target(14.178058624267578, 17.600414276123047, -1.077511191368103, 1.0, Face[Face([0.9995405767923673, -0.03030899776603807, 0.0], [15.177599201059945, 17.57010527835701], 2.309401076758503, 2.0), Face([0.5260186504248185, 0.8504730327325215, 0.0], [14.704077274692397, 18.450887308855567], 2.309401076758503, 2.0), Face([-0.47352192636754886, 0.8807820304985596, 0.0], [13.704536697900028, 18.481196306621605], 2.309401076758503, 2.0), Face([-0.9995405767923673, 0.030308997766038414, 0.0], [13.178518047475212, 17.630723273889085], 2.309401076758503, 2.0), Face([-0.526018650424819, -0.8504730327325212, 0.0], [13.65203997384276, 16.749941243390527], 2.309401076758503, 2.0), Face([0.47352192636754875, -0.8807820304985596, 0.0], [14.651580550635128, 16

In [ ]:
# policy = MDMA.solve_single_robot(model)

# Will need a grid for each robot

solution = solve_sequential(multi_problem)

Processing Robot 1
[Iteration 1   ] residual:        221 | iteration runtime: 206921.770 ms, (       207 s total)


In [13]:
paths = Vector{Vector{MDMA.MDPState}}(undef, 0)
for sol in solution.elements
    push!(paths, sol[2])
end
draw_frames(RenderConf(50, 4, true, false), multi_configs, paths)


LoadError: UndefVarError: solution not defined

In [14]:
draw_frames(RenderConf(50, 4, true, true), model, path)

LoadError: UndefVarError: model not defined

In [15]:
using PyPlot
path = paths[0]
#p1=scatter3D([],0)
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
x = zeros(0)
y = zeros(0)
z = zeros(0)
h = zeros(0)
for i in path
    append!( x, i.state.x )
    append!( y, i.state.y )
    append!( z, 7 )
    append!( h, dirAngle(i.state.heading) )
end
print("x:",x,"\ny:",y,"\nz:",z)
for i in 1:3
    tx = zeros(0)
    ty = zeros(0)
    tz = zeros(0)
    for t in target_trajectories[:,i]
        append!( tx, t.x )
        append!( ty, t.y )
        append!( tz, 0 )
    end
    print("\ntx:",tx,"\nty:",ty,"\ntz:",tz)
    ax.plot(tx,ty,tz)
end

ax.plot(x,y,z)

LoadError: BoundsError: attempt to access 0-element Vector{Vector{MDPState}} at index [0]

In [16]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]))
print(h)


for i in 1:20
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    location = [path[i].state.x; path[i].state.y; 7]
    camera_heading = [cos(dirAngle(path[i].state.heading)); sin(dirAngle(path[i].state.heading)); -sin(pitch)]
    print(path[i].state.heading, " ", location, " ", location+camera_heading, "\n")
    ax.scatter(path[i].state.x, path[i].state.y, 7, color="red")
    ax.scatter(path[i].state.x+camera_heading[1], path[i].state.y+camera_heading[2], 7+camera_heading[3], color="blue")
    ax.scatter(target_trajectories[i,1].x, target_trajectories[i,1].y, 0)
    ax.scatter(target_trajectories[i,2].x, target_trajectories[i,2].y, 0)
    ax.scatter(target_trajectories[i,3].x, target_trajectories[i,3].y, 0)
end

82.4002527426068 61.9527999101565

LoadError: UndefVarError: h not defined

In [17]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]), "\n")
for w in [0, sensor.resolution[1]]
    for h in [0, sensor.resolution[2]]
        point = inv(sensor.extrinsics)*inv(sensor.intrinsics)*[w;h;1]
        print(w, " ", h, " ", point, "\n")
    end
end

82.4002527426068 61.9527999101565
0.0 0.0 [0.6003001500750377, 0.8754377188594297, -1.0]
0.0 2160.0 [-0.6003001500750375, 0.8754377188594297, -1.0]
3840.0 0.0 [0.6003001500750377, -0.8754377188594297, -1.0]
3840.0 2160.0 [-0.6003001500750375, -0.8754377188594297, -1.0]


In [18]:
fov = [2*atan(sensor.resolution[1], 2*focal_length[1]); 2*atan(sensor.resolution[2], 2*focal_length[2])]
print(rad2deg(fov[1]), " ", rad2deg(fov[2]), "\n")
for w in [0, sensor.resolution[1]]
    for h in [0, sensor.resolution[2]]
        point = inv(sensor.extrinsics)*inv(sensor.intrinsics)*[w;h;1]
        print(w, " ", h, " ", point, "\n")
    end
end

82.4002527426068 61.9527999101565
0.0 0.0 [0.6003001500750377, 0.8754377188594297, -1.0]
0.0 2160.0 [-0.6003001500750375, 0.8754377188594297, -1.0]
3840.0 0.0 [0.6003001500750377, -0.8754377188594297, -1.0]
3840.0 2160.0 [-0.6003001500750375, -0.8754377188594297, -1.0]
